In [1]:
from dotenv import load_dotenv  
import pandas as pd
from woocommerce import API
import os

In [2]:
load_dotenv()
url = os.getenv("URL")
consumer_key = os.getenv("CONSUMER_KEY")
consumer_secret = os.getenv("CONSUMER_SECRET")


In [3]:
wcapi = API(
    url=url,
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    wp_api=True,
    version="wc/v3"
)

In [4]:
datos = wcapi.get("orders?per_page=100").json()


In [5]:
# row = []

# for items in datos:

#     if items['status'] != "on-hold":
#         continue

#     id = items['id']
#     nombre_completo = items['billing']['first_name'] + " " + items['billing']['last_name']
#     address = items['billing']['address_1']
#     distrito = items['billing']['distrito']
#     provincia = items['billing']['provincia'] 
#     departamento = items['billing']['departamento']
#     monto = items['total']
#     phone = items['billing']['phone']

    
#     row.append([id,nombre_completo,address,distrito,provincia,departamento,monto,phone])








In [6]:
def payment_name(method_pay):
    if method_pay == 'woo-mercado-pago-basic':
        return 'Mercado Pago'
    if method_pay == 'micuentawebstd':
        return 'Pago Link - Izipay'
    else:
        return 'Revisar'
    

#'_billing_dni'
def get_meta_data(key,data):
    for meta_item in data['meta_data']:
        if meta_item['key'] == key:
            return meta_item['value']
    return None


def get_payment_id(method_pay,data):
    if method_pay == 'woo-mercado-pago-basic':
        return get_meta_data('_Mercado_Pago_Payment_IDs',data)
    if method_pay == 'micuentawebstd':
        return get_meta_data('Transaction ID',data)
    else:
        return 'Revisar'

In [7]:
row = []
    # order-shipped
    # if items['status'] != "on-hold":


for items in datos:

    if items['status'] != "processing":
        continue

    id = items['id']
    status = items['status']
    registro = items['date_created'].split('T')[0]
    nombre_completo = items['billing']['first_name'] + " " + items['billing']['last_name']
    address = items['billing']['address_1']
    distrito = items['billing']['distrito']
    provincia = items['billing']['provincia'] 
    departamento = items['billing']['departamento']
    monto = items['total']
    phone = items['billing']['phone'].replace(" ","")
    sku = items['line_items'][0]['sku']
    email = items['billing']['email']
    payment_method = payment_name(items['payment_method'])
    dni = get_meta_data('_billing_dni',items)
    payment_number = str(get_payment_id(items['payment_method'],items))
    
    row.append([id,status,registro,dni, nombre_completo,'-','-','-','-','-',address,'-',distrito,provincia,departamento,sku,monto,phone[-9:],'-','-','1',email,'-',payment_method,'Capturado',payment_number])

In [8]:
df = pd.DataFrame(row)
df = df.sort_values(by=0, ascending=True)
df

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
9,9441,processing,2024-04-02,75201936,Alison Mendoza,-,-,-,-,-,...,1311.45,945919968,-,-,1,alisonmendoza07@gmail.com,-,Mercado Pago,Capturado,75248049263
8,9442,processing,2024-04-02,75343926,Angello Rodas,-,-,-,-,-,...,1941.45,940650285,-,-,1,awtzleko@gmail.com,-,Mercado Pago,Capturado,75253412051
7,9448,processing,2024-04-03,73538875,Jeremias Melo Huamán,-,-,-,-,-,...,1549.00,627303486,-,-,1,jeremiasmelo13@gmail.com,-,Pago Link - Izipay,Capturado,430796
6,9461,processing,2024-04-03,71597134,Claudio Arturo Curimnaya Gil,-,-,-,-,-,...,2599.00,992261999,-,-,1,claudiosdsk@gmail.com,-,Pago Link - Izipay,Capturado,120694
5,9463,processing,2024-04-03,77243811,Karla Miyan,-,-,-,-,-,...,2728.95,992184833,-,-,1,estefan.ll@hotmail.com,-,Mercado Pago,Capturado,75330586231
4,9468,processing,2024-04-04,72201175,Milene Isabel Dolci Ríos,-,-,-,-,-,...,1549.00,939207622,-,-,1,dolcimilene@gmail.com,-,Pago Link - Izipay,Capturado,586925
3,9476,processing,2024-04-04,77702043,Samira Cordero,-,-,-,-,-,...,1399.00,907846249,-,-,1,magalyrociobravofernandez@gmail.com,-,Pago Link - Izipay,Capturado,009811
2,9478,processing,2024-04-04,46790190,Lisette Limache,-,-,-,-,-,...,1899.00,977185256,-,-,1,lisettel@outlook.es,-,Pago Link - Izipay,Capturado,052631
1,9483,processing,2024-04-04,71656411,sara paucar rojas,-,-,-,-,-,...,2549.00,977749458,-,-,1,paucarrojassara@gmail.com,-,Pago Link - Izipay,Capturado,085249
0,9488,processing,2024-04-05,70069848,José Antonio,-,-,-,-,-,...,549.00,900027422,-,-,1,youngkrony2018@gmail.com,-,Pago Link - Izipay,Capturado,192141


In [9]:
if os.path.exists('reporte.csv'):
    os.remove('reporte.csv')
    df.to_csv('reporte.csv', sep=',', index=False, encoding='utf-8-sig')

else:
    df.to_csv('reporte.csv', sep=',', index=False, encoding='utf-8-sig')

    
